In [1]:
import time

import pandas as pd
import dotenv
import os
import sys
import datetime
sys.path.append('./_01_Simulation/')

import _01_Simulation.LLM_Edge_Usability
import _01_Simulation.LLM_Dynamic_Weights

In [2]:
# load .env file
dotenv.load_dotenv()

True

# 1. Edge usability

In [12]:
# Params (currently only considers openai models
approaches = ["zeroshot", "fewshot"]

In [13]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset.csv'), sep=";")

In [14]:
df_evaluation.head()

,action,edgeUsable
0,A person fell on the ground.,False
1,There is an unexpected construction on the usu...,False
2,Fallen branches from nearby trees.,False
3,Street cleaning crew sweeping debris.,False
4,The sidewalk is obstructed by a parked car.,False


In [15]:
# TODO
df_evaluation = df_evaluation.iloc[:10]

In [16]:
# Invoke GPT-3.5 to obtain the predictions for edge-usability

for approach in approaches:
    predictions = []
    for index, row in df_evaluation.iterrows():
        model_output = _01_Simulation.LLM_Edge_Usability.invoke_llm(row["action"], model_type="openai", approach=approach)
        prediction = _01_Simulation.LLM_Edge_Usability.parse_response(model_output)
        predictions.append(prediction)
    
    df_evaluation["prediction"] = predictions
    df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-edge-usability-openai-{approach}-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]
/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  edges_list = [(f'{row[0]}', f'{row[1]}') for _, row in df.iterrows()]
/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Edge_Usability.py:397: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

# 2. Dynamic Edge Weights

In [30]:
# Params (currently only considers openai models
approaches = ["zeroshot", "fewshot"]

In [31]:
# load evaluation csv
df_evaluation = pd.read_csv(os.path.join(os.getenv('RESOURCES'), 'EvaluationDataset-dynamic-edge-weight.csv'), sep=";")

In [32]:
df_evaluation.head()

,action,lengthDependency,LLMAnswer
0,A person fell on the ground.,False,False
1,There is an unexpected construction on the usu...,True,True
2,Fallen branches from nearby trees.,False,False
3,Street cleaning crew sweeping debris.,False,False
4,The sidewalk is obstructed by a parked car.,False,False


In [33]:
# TODO
df_evaluation = df_evaluation.iloc[:10]

In [34]:
df_evaluation

,action,lengthDependency,LLMAnswer
0,A person fell on the ground.,False,False
1,There is an unexpected construction on the usu...,True,True
2,Fallen branches from nearby trees.,False,False
3,Street cleaning crew sweeping debris.,False,False
4,The sidewalk is obstructed by a parked car.,False,False
5,Heavy rain has flooded the pathway.,True,True
6,A large pothole has formed in the middle of th...,False,True
7,Traffic cones block the entrance to the path.,True,False
8,A fallen power line is lying across the path.,False,False
9,The pathway is covered in thick mud due to rec...,True,True


In [35]:
# Invoke GPT-3.5 to obtain the predictions for dynamic-edge-weights

for approach in approaches:
    predictions = []
    result_types = []
    for index, row in df_evaluation.iterrows():
        model_output, _, result_type = _01_Simulation.LLM_Dynamic_Weights.invoke_llm_chain(row["action"], model_type="openai", approach=approach)
        prediction = _01_Simulation.LLM_Dynamic_Weights.parse_output_weights(model_output)
        predictions.append(prediction)
        result_types.append(result_type)
    
    df_evaluation.to_csv(os.path.join(os.getenv('RESULTS'), f'eval-res-dynamic-openai-{approach}-{datetime.datetime.now().isoformat()}.csv'), sep=";", index=False)

/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Dynamic_Weights.py:752: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return edges_list


Affected for the whole length:  

    Are follow up questions needed here: Yes.
    Follow up: Would the event affect the accessibility of the whole edge?
    Intermediate answer: No, the event would only affect the accessibility of the edge at the specific point where the person fell on the ground.
    So the final answer is: The answer is False, the event doesn't affect the accessibility of the whole edge.
Minutes


/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Dynamic_Weights.py:752: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return edges_list


Affected for the whole length:  

    Are follow up questions needed here: Yes.
    Follow up: Would a vehicle be affected by the event for the whole length of the edge?
    Intermediate answer: Yes, an unexpected construction on the usual path would affect the vehicle for the whole length of the edge.
    So the final answer is: The answer is True, the event affects the whole edge.
Factor


/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Dynamic_Weights.py:752: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return edges_list


...

Affected for the whole length:  
    Are follow up questions needed here: No.
    Intermediate answer: Yes, traffic cones blocking the entrance to the path would affect the accessibility of the whole edge.
    So the final answer is: The answer is True, the event affects the whole edge.
Factor


/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Dynamic_Weights.py:752: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return edges_list


Affected for the whole length:  

Are follow up questions needed here: Yes.
Follow up: Would a vehicle be affected by the event for the whole length of the edge?
Intermediate answer: Yes, a fallen power line lying across the path would seriously affect a vehicle for the whole length of the edge.
So the final answer is: The answer is True, the event affects the whole edge.
Factor


/Users/paulkoenig/WebstormProjects/medicar/_01_Simulation/LLM_Dynamic_Weights.py:752: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return edges_list


Affected for the whole length:  

    Are follow up questions needed here: Yes.
    Follow up: Would a vehicle be affected by the event for the whole length of the pathway?
    Intermediate answer: Yes, the pathway being covered in thick mud due to recent rain would seriously affect a vehicle for the whole length of the pathway.
    So the final answer is: The answer is True, the event affects the whole edge.
Factor
